In [1]:
import torch.nn as nn
import numpy as np

import torch
import random

from tqdm import tqdm
from useful_functions import state_evol, random_state, fidelity
from data import generates_data

In [2]:
# The first step in our project should be specify the physical system that we are going to work on. For this, we require define the Hamiltonian (i.e. the physical system) and,
# for that, we require the Pauli basis.

sigma_x = np.array([[0, 1], 
                    [1, 0]], dtype=complex)

sigma_y = np.array([[0, -1j], 
                    [1j, 0]], dtype=complex)

sigma_z = np.array([[1, 0], 
                    [0, -1]], dtype=complex)

pauli_basis = [np.eye(2, dtype=complex), sigma_x, sigma_y, sigma_z]

We choose a simply Hamiltonian for this project, but feel free to use anyone. Just remember, for a different Hamiltonian is necesarry to train again the model.

\begin{equation}
    H = \frac{1}{\hbar} \left( \sigma_x \otimes \sigma_x + \sigma_z \otimes \sigma_z \right).
\end{equation}

In [3]:
H = np.kron(pauli_basis[1], pauli_basis[1]) + np.kron(pauli_basis[3], pauli_basis[3])

In [4]:
# In this part we define the architecture for our model. We require two channel inputs for convolutional layers and, in the latent space, we introduce a scalar parameter.
# The output is a two channel image that represents the output state, real and imaginary part.

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=2, out_channels=8, kernel_size=2)  # Output: 8 x 3 x 3
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=2)  # Output: 16 x 2 x 2
        
        # Fully connected layers
        self.fc1 = nn.Linear(16 * 2 * 2 + 1, 32)  # Flattened vector + scalar parameter
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 4 * 4 * 2)  # Output for the 4x4x2 image
        
    def forward(self, x, scalar_param):
        # Convolutional layers with Tanh activation
        x = torch.tanh(self.conv1(x))  # Output: 8 x 3 x 3
        x = torch.tanh(self.conv2(x))  # Output: 16 x 2 x 2
        
        # Flatten the output of the convolutional layer
        x = x.flatten()  # Flatten: 16*2*2 = 64
        x = torch.cat((x, scalar_param))  # Concatenate: 64 + 1 = 65
        
        # Fully connected layers with Tanh activation
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        
        # Output layer
        x = torch.tanh(self.fc3(x))
        
        # Reshape to obtain the output with size 4x4x2
        x = x.view(2, 4, 4)
        
        return x
    
model = Net()

In [5]:
# For train our model an optimizer and matrics must be specified

def metric(predicted, target):
    rho_model   = predicted[0] + 1j*predicted[1]
    rho_target  = target[0] + 1j*target[1]
    rho_target  = rho_target

    U, S, Vh    = torch.linalg.svd(rho_model)
    SP          = U @ torch.diag(torch.complex(S, torch.zeros_like(S))) @ torch.conj(U).T
    SP          = SP/torch.trace(SP)
    
    return 1 - fidelity(SP, rho_target)

optimizer   = torch.optim.Adam(model.parameters(), lr=1e-3)

One of the problems of Machine Learning with Quantum Information is that the outcomes states aren't perfectly satisfies the requirements for to be quantum states, that is, trace one and semi-positivity. This is because using hard-constraint to impose those conditions not secure to be satisfies at 100%.

A solution for that is consider a polar descomposition for the Neural Network output and, then, apply a soft-contrain for those requirements, such as renormalization for impose unitary trace.

In [6]:
# Define the parameters for training, that is, 
#   N_train for the number of iterations for train our model
#   N_data for the number of generated data for each train interation
#   We also define a loss_train list for append the loss function values for analyze the convergence of our model

N_train = 250
N_data  = 1000

loss_train = []

In [ ]:
# Ensure that n_qubits = 2 for the Hamiltonian system related to a two-qubit state

for train in tqdm(range(N_train), desc="Training Epochs"):

    # Generate the training data for this epoch
    input_data, output_data = generates_data(N_data=N_data, n_qubits=2, Hamiltonian=H)
    
    # Initialize loss for this epoch
    train_loss = 0

    # Loop through each data point in the generated dataset
    for n in range(N_data):
        model.train()  # Set the model to training mode
        optimizer.zero_grad()  # Reset gradients for this batch

        # Forward pass: predict the output using the model
        prediction = model(input_data[0][n], input_data[1][n])  # input_data[0] are states, input_data[1] are times
        targets = output_data[n]  # Target values are the final states

        # Calculate the loss
        loss = metric(prediction, targets)

        # Backpropagation: compute gradients and update the model parameters
        loss.backward()
        optimizer.step()

        # Accumulate the training loss for this epoch
        train_loss += loss.item() / N_data

    # Store the average loss for this epoch
    loss_train.append(train_loss)

    # Optionally, print or log training loss for this epoch
    # print(f"Epoch {train+1}/{N_train}, Loss: {train_loss:.4f}")

In [11]:
torch.save(model.state_dict(), "model.pth")